<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/bgRemoval/tests/PaddleSeg_matting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaddleSeg - matting + BG-inpainting

[GitHub](https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.5/Matting)


## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [1]:
!nvidia-smi

Mon May  2 09:34:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [2]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='bgRemoval',library='PaddleSeg')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2480  100  2480    0     0  11221      0 --:--:-- --:--:-- --:--:-- 11272
what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## 3. clone GIT repo

In [4]:
import os
root_path = '/content/PaddlesSeg'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/PaddlePaddle/PaddleSeg {root_path}

%ls

Cloning into '/content/PaddlesSeg'...
remote: Enumerating objects: 17638, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 17638 (delta 53), reused 92 (delta 35), pack-reused 17518
Receiving objects: 100% (17638/17638), 341.87 MiB | 33.56 MiB/s, done.
Resolving deltas: 100% (11256/11256), done.
database/  database_mod.py  PaddlesSeg/  __pycache__/  sample_data/


## 4. Setting up the environment

In [6]:
# installing PaddleSeg
%cd {root_path}
!pip install -r requirements.txt
!pip install -e .


# installing Matting
%cd Matting
!pip install -r requirements.txt

/content/PaddlesSeg
Obtaining file:///content/PaddlesSeg
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
/content/PaddlesSeg/Matting
     |████████████████████████████████| 60.3 MB 1.5 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
